# מבוא להנדסת פרומפטים
הנדסת פרומפטים היא התהליך של עיצוב ואופטימיזציה של פרומפטים למשימות עיבוד שפה טבעית. זה כולל בחירה נכונה של פרומפטים, כיוונון הפרמטרים שלהם, והערכת הביצועים שלהם. הנדסת פרומפטים היא חיונית להשגת דיוק ויעילות גבוהים במודלים של עיבוד שפה טבעית. בחלק זה נחקור את היסודות של הנדסת פרומפטים באמצעות המודלים של OpenAI לצורך חקירה.


### תרגיל 1: טוקניזציה
חקור טוקניזציה באמצעות tiktoken, טוקנייזר מהיר בקוד פתוח של OpenAI  
ראה דוגמאות נוספות ב-[OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst).


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### תרגיל 2: בדיקת הגדרת מפתח Github Models

הרץ את הקוד למטה כדי לוודא שה-Endpoint של Github Models שלך מוגדר כמו שצריך. הקוד פשוט בודק פקודה בסיסית ומוודא את ההשלמה. הקלט `oh say can you see` אמור להיות מושלם בסגנון של `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### תרגיל 3: המצאות  
בדקו מה קורה כאשר אתם מבקשים מה-LLM להחזיר תשובות עבור נושא שאולי לא קיים, או עבור נושאים שהוא אולי לא מכיר כי הם מחוץ למאגר הנתונים עליו הוא אומן (נתונים עדכניים יותר). שימו לב איך התגובה משתנה אם תנסו ניסוח שונה, או מודל אחר.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### תרגיל 4: מבוסס הוראות
השתמש במשתנה "text" כדי להגדיר את התוכן העיקרי  
ובמשתנה "prompt" כדי לתת הוראה שקשורה לתוכן העיקרי.

כאן אנחנו מבקשים מהמודל לסכם את הטקסט עבור תלמיד בכיתה ב'


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### תרגיל 5: פרומפט מורכב
נסו בקשה שמכילה הודעות של מערכת, משתמש ועוזר  
המערכת קובעת את ההקשר של העוזר  
הודעות המשתמש והעוזר יוצרות הקשר של שיחה מרובת פניות

שימו לב כיצד האישיות של העוזר מוגדרת כ"סארקסטית" בהקשר של המערכת.  
נסו להשתמש בהקשר אישיות שונה. או נסו סדרה אחרת של הודעות קלט/פלט


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### תרגול: חקור את האינטואיציה שלך
הדוגמאות למעלה נותנות לך דפוסים שאפשר להשתמש בהם כדי ליצור פקודות חדשות (פשוטות, מורכבות, הוראות וכו') - נסה ליצור תרגולים נוספים כדי לחקור רעיונות נוספים שדיברנו עליהם, כמו דוגמאות, רמזים ועוד.



---

Certainly! Here is your text translated into Hebrew:

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לשים לב כי תרגומים אוטומטיים עלולים להכיל שגיאות או אי-דיוקים. המסמך המקורי בשפת המקור ייחשב כמקור הסמכותי. למידע קריטי, מומלץ לפנות לתרגום מקצועי על ידי אדם. איננו אחראים לכל אי-הבנה או פרשנות שגויה הנובעת מהשימוש בתרגום זה.
